<a href="https://colab.research.google.com/github/daviddkovacs/MOGPR-time-series-reconstruction/blob/main/MOGPR_data_fusion_reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing required libraries

In [ ]:
from google.colab import drive
import ee
ee.Authenticate()

!pip install xesmf
!pip install wxee
!pip install rasterio
!pip install geopandas
!pip install geextract
!pip install gmaps
!pip install pdf2image
!pip install mogptk
!pip install GPy
!pip install openeo
!pip install PyPDF2

ee.Initialize()

In [3]:
from geextract import ts_extract, relabel, date_append, dictlist2sqlite
from google.colab import files
import matplotlib.pyplot as plt
from datetime import datetime
import osgeo.ogr
import json
from pdf2image import convert_from_path
import sys
import pickle
import GPy
import openeo
import h5py
import math
from scipy.stats import pearsonr
import csv
import PyPDF2
from shapely.geometry import box
import pandas as pd
import numpy as np
from openeo.rest.conversions import timeseries_json_to_pandas
import itertools
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import time
import os
import gmaps
import seaborn as sns
import matplotlib.ticker as ticker
import traceback
from datetime import timedelta
import shapely
from shapely.geometry import mapping
from scipy.stats.stats import pearsonr, spearmanr
from sklearn.metrics import mean_absolute_error

Define temporal range


if you define, in the first cells, the temporal domain to be the same, i.e. 2019 for S3 and 2019 for MODIS, it will be gap filling

if you define MODIS to be, let's say, 2010-2020 and S3 to be only 2019, then it will reconstruct/extend S3 to the broader, 2010-20 timeframe
i.e temporally reconstruct.

In [4]:
start_date = datetime(2000, 1, 1)
end_date   = datetime(2022, 1, 1)

# Available dates for S3 based vegetation traits: 2017-01-01 to 2021-12-01
start_date_S3 = datetime(2019, 1, 1)
end_date_S3   = datetime(2020, 1, 1)

Tstep      = 8

Define spatial domain

Adjust site as needed. This site is defaulted to Valencia Anchor Station

In [5]:
site ={'geometry':{'coordinates':
         [[[-1.296402507606762, 39.57334640048957],
          [-1.296402507606762, 39.56600234855508],
          [-1.2870469625628167, 39.56600234855508],
          [-1.2870469625628167, 39.57334640048957]]]
                     },u'type': u'Polygon'}



Here we create the 8 day temporal composites of MODIS NDVI. This dataset will be used as a guiding, predictor variable later on.

In [6]:
daily_collection = (
    ee.ImageCollection("MODIS/MOD09GA_006_NDVI")
    .filterDate(start_date, end_date)
)

# Create a list of dates at 8-day intervals
dates = ee.List.sequence(
    ee.Date(start_date).millis(),
    ee.Date(end_date).advance(1, 'day').millis(),  # Add 1 day to include the end_date
    8 * 24 * 60 * 60 * 1000  # 8 days in milliseconds
)

# function to create an 8-day mean composite
def create_8_day_composite(start_date):
    start_date = ee.Date(start_date)
    end_date = start_date.advance(8, 'day')

    mean_image = (
        daily_collection
        .filterDate(start_date, end_date)
        .mean()
    )
    system_id = start_date.format('YYYY_MM_dd')

    mean_image = mean_image.set('system:time_start', start_date)

    return mean_image.set('system:id', system_id)

composite_collection = ee.ImageCollection(dates.map(create_8_day_composite))

In [7]:
def create_empty_2D_list(nrows,ncols):

    empty_list = []
    for j in range(nrows):
        column = []
        for i in range(ncols):
            column.append(0)
        empty_list.append(column)
    return empty_list

def S3TOA_get_date(id):

    date_info = datetime.date(datetime(int(id[0:4]),
                                       int(id[4:6]),
                                       int(id[6:8])))
    return date_info


def MODIS_get_date(id):

    date_info = datetime.date(datetime(int(id[0:4]), int(id[5:7]), int(id[8:])))

    return date_info

def MODISNDVI_get_date(id):

    day_from_start = start_date + timedelta(days=int(id)*8)
    date_info = day_from_start

    return date_info


def extract_S3TOA_time_series(S3_LCC_flag,
                              S3_LAI_flag,
                              S3_FAPAR_flag,
                              S3_FVC_flag,
                              feature,start_date = [],end_date = [],stats='mean',pix_res=500
                              ):

    global product_name
    global exp_var_unit
    global exp_variable
    global plotylim_min
    global plotylim_max

    if S3_LCC_flag:

        product_name = 'projects/ee-dkvcsdvd/assets/EVT_regional/LCC_10_sites'
        band_name = ['LCC_GREEN']
        bands_name_out = ['LAI']
        exp_variable = "LCC"
        exp_var_unit = "$\mu$g/cm$^2$"
        plotylim_min = -7
        plotylim_max = 100

    if S3_LAI_flag:

        product_name = 'projects/ee-dkvcsdvd/assets/Paper3_GBOV/S3_LAI'
        band_name = ['LAI_GREEN']
        bands_name_out = ['LAI']
        exp_variable = "LAI"
        exp_var_unit = "m$^2$/m$^2$"
        plotylim_min = -0.5
        plotylim_max = 9

    if S3_FAPAR_flag:

        product_name = 'projects/ee-dkvcsdvd/assets/Paper3_GBOV/S3_FAPAR'
        band_name = ['FAPAR_GREEN']
        bands_name_out = ['LAI']
        exp_variable = "FAPAR"
        exp_var_unit = "[-]"
        plotylim_min = 0
        plotylim_max = 1

    if S3_FVC_flag:

        product_name = 'projects/ee-dkvcsdvd/assets/Paper3_GBOV/S3_FVC'
        band_name = ['FVC_GREEN']
        bands_name_out = ['LAI']
        exp_variable = "FVC"
        exp_var_unit = "[-]"
        plotylim_min = 0
        plotylim_max = 1

    geometry = ee.Geometry.Polygon(feature['geometry']['coordinates'])
    coll = ee.ImageCollection(product_name)\
        .filterBounds(geometry)\
        .filterDate(start_date_S3, end_date_S3)\
        .select(band_name,bands_name_out)

    images_list = [item.get('id') for item in coll.getInfo().get('features')]

    info = coll.getRegion(geometry,pix_res).getInfo()

    df=pd.DataFrame(info[1:],columns=info[0]).groupby('id').mean().reset_index()

    df['date']=df['id'].apply(lambda x: S3TOA_get_date(x))

    return df,images_list

def extract_MODISLAI_time_series(feature,start_date = [],end_date = [],stats='mean',pix_res=500):

    product_name = 'MODIS/061/MCD15A3H'
    band_name = ['Lai']
    bands_name_out = ['LAI']

    geometry = ee.Geometry.Polygon(feature['geometry']['coordinates'])

    coll = ee.ImageCollection(product_name)\
        .filterBounds(geometry)\
        .filterDate(start_date, end_date)\
        .select(band_name,bands_name_out)

    images_list = [item.get('id') for item in coll.getInfo().get('features')]

    info = coll.getRegion(geometry,pix_res).getInfo()

    df=pd.DataFrame(info[1:],columns=info[0]).groupby('id').mean().reset_index() # Used to be .mean()

    df['date']=df['id'].apply(lambda x: MODIS_get_date(x))

    return df,images_list

def extract_MODISFAPAR_time_series(feature,start_date = [],end_date = [],stats='mean',pix_res=500):

    product_name = 'MODIS/061/MCD15A3H'
    band_name = ['Fpar']
    bands_name_out = ['FAPAR']

    geometry = ee.Geometry.Polygon(feature['geometry']['coordinates'])

    coll = ee.ImageCollection(product_name)\
        .filterBounds(geometry)\
        .filterDate(start_date, end_date)\
        .select(band_name,bands_name_out)

    images_list = [item.get('id') for item in coll.getInfo().get('features')]

    info = coll.getRegion(geometry,pix_res).getInfo()

    df=pd.DataFrame(info[1:],columns=info[0]).groupby('id').mean().reset_index()

    df['date']=df['id'].apply(lambda x: MODIS_get_date(x))

    return df,images_list


def extract_MODISNDVI_time_series(feature,start_date = [],end_date = [],stats='mean',pix_res=500):

    product_name = composite_collection
    band_name = ['NDVI']
    bands_name_out = ['NDVI']

    geometry = ee.Geometry.Polygon(feature['geometry']['coordinates'])

    coll = ee.ImageCollection(product_name)\
        .filterBounds(geometry)\
        .filterDate(start_date, end_date)\
        .select(band_name,bands_name_out)

    images_list = [item.get('id') for item in coll.getInfo().get('features')]

    info = coll.getRegion(geometry,pix_res).getInfo()

    df=pd.DataFrame(info[1:],columns=info[0]).groupby('id').mean().reset_index() # Used to be mean but it's daily data an error when accumulating elements

    df['date']=df['id'].apply(lambda x: MODISNDVI_get_date(x))

    return df,images_list

def need_to_name(df_in):
    df_in['LAI'] = df_in.LAI
    return df_in

def need_to_name_CGLS_LAI(df_in):
    df_in['b1'] = df_in.b1 /31.875
    return df_in

def need_to_name_CGLS(df_in):
    df_in['b1'] = df_in.b1 /255
    return df_in

def need_to_name_merisdivLCC(df_in):
    df_in['b1'] = (df_in.b1) / 100
    return df_in

def need_to_name_gbov(df_in):
    df_in['b1'] = df_in.b1
    return df_in

def need_to_name_modisdivLCC(df_in):
    df_in['b1'] = (df_in.b1) / 2
    return df_in

def need_to_name_modisdivLAI(df_in):
    df_in['LAI'] = (df_in.LAI) / 10
    return df_in

def need_to_name_modisdivFAPAR(df_in):
    df_in['FAPAR'] = (df_in.FAPAR) / 100
    return df_in

def need_to_name_modisdivNDVI(df_in):
    df_in['NDVI'] = (df_in.NDVI)
    return df_in


def datestr_to_number(time_vec):

    """
    Function coverting date vector from string to absolute number vector
    """

    time_vec_num = np.asarray([ _.toordinal() for _ in time_vec], dtype=np.float32)
    return time_vec_num

def number_to_datestr(time_vec_num):

    """
    Function coverting date vector from string to absolute number vector
    """

    time_vec = [datetime.fromordinal(int(_)) for _ in time_vec_num]
    return time_vec

def GPY_retrieval_Noutput(DATA,TIME,Master_Ind,output_timevec,Nt,proc_line_print ='MOGPR modelling..',W_rank=1, trained_model=[]):
    """
    Function performing the multioutput gaussian-process regression at pixel level for gapfilling and temporal reconstruction purposes

    Args:
        DATA [array] : 3D (2DSpace, Time) array containing data to be processed
        TIME [array] : vector containing the dates of each layer in the time dimension
        Master_Ind [int] : Index identifying the Master output
        output_timevec [array] :vector containing the dates on which output must be estimated
        Nt [int]  : # of time the GP training must be performed (def=1)

    """

    noutput_timeseries = len(DATA)

    x_size = DATA[0].shape[1]
    y_size = DATA[0].shape[2]
    imout_sz = (output_timevec.shape[0],x_size,y_size)

    Xtest     = output_timevec.reshape(output_timevec.shape[0],1)
    Out_QFlag = np.ones((x_size,y_size), dtype=bool)
    Out_mean  = []
    Out_unc   = []
    Out_model = create_empty_2D_list(x_size,y_size)

    for _ in range(noutput_timeseries):
        Out_mean.append(np.full(imout_sz,np.nan))
        Out_unc.append(np.full(imout_sz,np.nan))

    model_parameter_names = None
    cnt = 0
    tot = x_size*y_size
    check_param_names_flag= True

    for x, y in itertools.product(range(x_size), range(y_size)):

            X_vec = []
            Y_vec = []
            Y_mean_vec = []
            Y_std_vec  = []

            for ind in range(noutput_timeseries):

              #X_tmp  = np.array(TIME[ind],dtype=np.float128)
              #Y_tmp  =np.array( DATA[ind][:,x,y],dtype=np.float128)
              X_tmp  = TIME[ind]
              Y_tmp  = DATA[ind][:,x,y]
              X_tmp  = X_tmp[~np.isnan(Y_tmp),np.newaxis]
              Y_tmp  = Y_tmp[~np.isnan(Y_tmp),np.newaxis]
              X_vec.append(X_tmp)
              Y_vec.append(Y_tmp)
              del X_tmp,Y_tmp


            if np.size(Y_vec[Master_Ind]) >0:

                # Data Normalization
                for ind in range(noutput_timeseries):
                    Y_mean_vec.append(np.mean(Y_vec[ind]))
                    Y_std_vec.append(np.std(Y_vec[ind]))
                    Y_vec[ind] = (Y_vec[ind]-Y_mean_vec[ind])/Y_std_vec[ind]

                # Multi-output training and testing sets
                Xtrain = X_vec
                Ytrain = Y_vec

                nsamples, npixels = Xtest.shape
                noutputs = len(Ytrain)

                for i_test in range(Nt):

                    Yp = np.zeros((nsamples, noutputs))
                    Vp = np.zeros((nsamples, noutputs))

                    K            =  GPy.kern.Matern32(Xtrain[0].shape[1])  # Use RBF or Matern32 as kernels
                    LCM          =  GPy.util.multioutput.LCM(input_dim    =  Xtrain[0].shape[1],
                                                             num_outputs  =  noutputs,
                                                             kernels_list =  [K] * noutputs, W_rank=W_rank)

                    model = GPy.models.GPCoregionalizedRegression(Xtrain, Ytrain, kernel=LCM.copy())  # , W_rank=noutputs)
                    model['.*Mat32.var'].constrain_fixed(1.)

                    if not np.isnan(Ytrain[1]).all():

                            try:
                                #if trained_model is None:
                                model.optimize()
                                #else:
                                    #here we should iterate through the model's parameters to assigned the pretrained values
                                   # model = 1

                                list_tmp = [model.param_array]

                                for _ in range(noutput_timeseries):
                                    #print('model.sum.ICM'+str(_)+'.B.B')
                                    list_tmp.append(eval('model.sum.ICM'+str(_)+'.B.B'))
                                Out_model[x][y]=list_tmp
                                # list_tmp contains [model.param_array, model.sum.ICM0.B.B,model.sum.ICM1.B.B])

                                if check_param_names_flag:
                                    model_parameter_names = model.parameter_names()
                                    check_param_names_flag=False
                            except:
                                Out_QFlag[x,y]=False
                                continue


                            for out in range(noutputs):
                                newX = Xtest.copy()

                                newX = np.hstack([newX, out * np.ones((newX.shape[0], 1))])
                                noise_dict = {'output_index': newX[:, -1:].astype(int)}
                                Yp[:,None, out],Vp[:,None,out] =model.predict(newX, Y_metadata=noise_dict)

                            if i_test==0:

                                for ind in range(noutput_timeseries):
                                    Out_mean[ind][:,None,x,y] = (Yp[:,None, 0]*Y_std_vec[ind]+Y_mean_vec[ind])/Nt
                                    Out_unc[ind][:,None,x,y]  = (Vp[:,None, 0]*Y_std_vec[ind])/Nt

                            else:
                                for ind in range(noutput_timeseries):
                                    Out_mean[ind][:,None,x,y] = Out_mean[ind][:,None,x,y] + (Yp[:,None, 0]*Y_std_vec[ind]+Y_mean_vec[ind])/Nt
                                    Out_unc[ind][:,None,x,y]  = Out_unc[ind][:,None,x,y]  + (Vp[:,None, 0]*Y_std_vec[ind])/Nt

                            del Yp,Vp
    if check_param_names_flag:
        print('\n')
        print("******************************************************************************************")
        print("No model has been trained within the processed block (probably due to non-valid input data)".upper())
        print("Processing is skippped to next block!".upper())
        print("******************************************************************************************")
        print('\n')

    return  Out_mean, Out_unc, Out_QFlag, Out_model,model_parameter_names

Here we are going to run the MOGPR function.


In [40]:

def main_loop(S3_LCC_flag = False, S3_FVC_flag = False, S3_LAI_flag= False,S3_FAPAR_flag= False,    # variable to MOGPR reconstruct
              MODISLAI_flag = False, MODISFAPAR_flag= False, MODISNDVI_flag= False):                # Ancillary variable

    feature = site

    TIME = []   # Time list containing the dates for the reconstructing (S3) and predictor (MODIS) variables
    DATA  = []  #  Variable values for reconstructing (S3) and predictor (MODIS) variables
    TIME_str = []
    legend_vec = []

    print('S3TOAGPR time series being retrieved')
    dfS3TOA,_ = extract_S3TOA_time_series(S3_LCC_flag,S3_LAI_flag,S3_FAPAR_flag,S3_FVC_flag,feature,start_date,end_date)
    dfS3TOA = need_to_name(dfS3TOA)
    TIME.append(datestr_to_number(dfS3TOA['date']))
    TIME_str.append(dfS3TOA['date'])
    DATA.append(np.reshape(dfS3TOA['LAI'].values,(dfS3TOA.shape[0],1,1)))
    legend_vec.append('S3TOA')

    if MODISLAI_flag:
        print('MODISLAI time series being retrieved')
        dfMODISLAI,_ = extract_MODISLAI_time_series(feature,start_date,end_date)
        dfMODISLAI = need_to_name_modisdivLAI(dfMODISLAI)
        TIME.append(datestr_to_number(dfMODISLAI['date']))
        TIME_str.append(dfMODISLAI['date'])
        DATA.append(np.reshape(dfMODISLAI['LAI'].values,(dfMODISLAI.shape[0],1,1)))
        legend_vec.append('MODIS')
        anc_variable = "LAI"

    if MODISFAPAR_flag:
        print('MODISFAPAR time series being retrieved')
        dfMODISFAPAR,_ = extract_MODISFAPAR_time_series(feature,start_date,end_date)
        dfMODISFAPAR = need_to_name_modisdivFAPAR(dfMODISFAPAR)
        TIME.append(datestr_to_number(dfMODISFAPAR['date']))
        TIME_str.append(dfMODISFAPAR['date'])
        DATA.append(np.reshape(dfMODISFAPAR['FAPAR'].values,(dfMODISFAPAR.shape[0],1,1)))
        legend_vec.append('MODIS')
        anc_variable = "FAPAR"


    if MODISNDVI_flag:
        print('MODISNDVI time series being retrieved')
        dfMODISNDVI,_ = extract_MODISNDVI_time_series(feature,start_date,end_date)
        dfMODISNDVI = need_to_name_modisdivNDVI(dfMODISNDVI)
        TIME.append(datestr_to_number(dfMODISNDVI['date']))
        TIME_str.append(dfMODISNDVI['date'])
        DATA.append(np.reshape(dfMODISNDVI['NDVI'].values,(dfMODISNDVI.shape[0],1,1)))
        legend_vec.append('MODIS')
        anc_variable = "NDVI"

    Nout = len(DATA)
    Nt =1

    time_vec_MIN        = np.min(list(pd.core.common.flatten(TIME)))
    time_vec_MAX        = np.max(list(pd.core.common.flatten(TIME)))
    output_timevec      = np.array(range(int(time_vec_MIN),int(time_vec_MAX),Tstep),dtype=float)
    output_time         = number_to_datestr(output_timevec)
    outputs_timevec_str =[ _.strftime("%Y%m%d") for _ in output_time]

    print('MULTIOUTPUT Being calculated, please wait')
    Master_Ind = 0
    Out_mean, Out_unc, Out_QFlag, Out_model,model_parameter_names = GPY_retrieval_Noutput(DATA[:],TIME[:],Master_Ind,output_timevec,
                                                                                                Nt,proc_line_print ='MOGPR modelling..')

    plt.figure(figsize= (25,6))
    sensor = legend_vec[0]

    number_sensor_inputs = len(DATA)
    for number in range(number_sensor_inputs):
        p1 = plt.plot_date(TIME_str[number],DATA[number].ravel(),"o",label = legend_vec[number])


    p2 = plt.plot_date(output_time, Out_mean[0].ravel(),"-",label = "S3_reconstructed")

    plt.fill_between(output_time, Out_mean[0].ravel()+Out_unc[0].ravel(),
                        Out_mean[0].ravel()-Out_unc[0].ravel(), alpha=0.5)
    fontsize = 40

    plt.yticks(fontsize=fontsize)
    plt.xticks(fontsize=fontsize)
    # plt.title(site_info[1]+" " + site_info[0]+"  (" +site_info[3]+", " + site_info[4]+")  ancillary : " +anc_variable ,fontsize=fontsize)
    plt.ylabel("",fontsize=fontsize)
    plt.ylim([plotylim_min,plotylim_max])
    plt.ylabel(exp_variable + " "+ exp_var_unit,fontsize=fontsize)
    plt.grid(True)
    plt.tight_layout()
    plt.legend(fontsize=20)
    plt.show()


    """
    Out_mean = mean estimate output from the MOGPR algorithm
    Out_unc = epistemic uncertainty associated with the MOGPR algorithm
    """


We call the function, use the flag associated with the variable-sensor you want to use:

For reconstruction: use S3 flags
For predictor/guiding variable: use MODIS flags


Here we are reconstructing S3 Based FVC with ancillary MODIS based FAPAR and NDVI time series.

In [ ]:
main_loop(S3_FVC_flag = True  # Reconstructing
          ,MODISFAPAR_flag = True # Predictor 1
          ,MODISNDVI_flag = False # Predictor 2
          )

S3TOAGPR time series being retrieved
MODISFAPAR time series being retrieved
MULTIOUTPUT Being calculated, please wait
